In [ ]:
from scipy import stats
import numpy as np
import pandas as pd

from scipy.stats import norm

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## Chi-square test

  - dados numéricos e discretos
  - comparação de 2 ou mais distribuições
    - p.ex. número de escamas de cobras fêmeas, de mesma espécie, em altitudes diferentes
    - são estatisticamente diferentes?
    - coletaremos 5 amostras a 1200 m e 5 amostras a 200 m

### Exemplo 01 - medicina

  - tabela de contingência

In [ ]:
# testes x estado de pacientes
cases = np.array([[10, 4], [3, 29]]).T
dfc = pd.DataFrame(cases)
dfc.columns = ['são', 'doente']
dfc.index = ["negativo", "positivo"]
dfc

### Exemplo 02 - ecologia ~ cobras

In [ ]:
chi2, pvalue, dof, expected = stats.chi2_contingency(dfc)
chi2, pvalue, dof, expected 

In [ ]:
# scales = observed data
scales = np.array([[16, 18, 25, 14, 12], [32, 24, 27, 35, 40]]).T
scales

In [ ]:
df = pd.DataFrame(scales)
df.columns = ['alt1200', 'alt200']
df.index = [32, 33, 34, 35, 36]
df

## Valores observados x Valores esperados

### Pearson

In [ ]:
dfa = df.copy()
dfa['tot_marg_escamas'] = dfa.sum(axis = 1)
dfa

In [ ]:
dfa.sum(axis = 0)

In [ ]:
pd.DataFrame(dfa.sum(axis = 0)).to_dict()

In [ ]:
pd.DataFrame(dfa.sum(axis = 0)).to_dict()[0]

In [ ]:
dfa.loc[len(dfa)] = pd.DataFrame(dfa.sum(axis = 0)).to_dict()[0]
lista = list(dfa.index)
lista[-1] = 'tot_marg_altitude'
dfa.index = lista
dfa

In [ ]:
dfp = dfa.copy()
nrow, ncol = dfp.shape
tot_animal = dfp.iloc[nrow-1, ncol-1]
dfp = dfp / tot_animal
print('tot animal', tot_animal)
dfp

### Valor experado: dfexp

  - Multiplicação das percentagens marginais coluna (j) x marginais linha (i) é a percentagem esperada de uma célula (i,j)

In [ ]:
dfexp = dfp.copy()

pos_tot_marg_escamas  = ncol - 1
pos_tot_marg_altitude = nrow - 1
print(pos_tot_marg_escamas, pos_tot_marg_altitude, "\n")

for i in range(nrow):
    for j in range(ncol):
        dfexp.iloc[i, j] = dfp.iloc[i, pos_tot_marg_escamas] * dfp.iloc[pos_tot_marg_altitude, j]
        
dfexp

In [ ]:
### Em termos de valores totais

In [ ]:
df_esperado = dfexp.copy()
(df_esperado*tot_animal).round(2)

In [ ]:
# os valores esperados
df_esperado = dfexp.copy()
df_esperado = (df_esperado*tot_animal)
df_esperado

In [ ]:
df_espval = df_esperado.copy()
df_espval = df_espval.iloc[:-1, :-1]
df_espval

In [ ]:
# os valores observados
df

In [ ]:
df - df_espval

In [ ]:
(df - df_espval)**2

### Estatística de chi2 

<p style="font-size:30px; color:yellow;">$\chi2 = \sum_{i=1}^{N}{\frac{(val_{esp,i}-val_{obs,i})**2}{val_{esp,i}}}$ </p>

In [ ]:
dfchi = (df - df_espval)**2 / df_espval
dfchi

In [ ]:
chi2 = np.sum(np.sum(dfchi, axis=0))
f"chi2 = {chi2:.3f}"

### Dataframe == tabela de contingência

In [ ]:
chi2, pvalue, dof, expected = stats.chi2_contingency(df)
chi2, pvalue, dof, expected 

In [ ]:
f"estatística de chi-quadrado {chi2:.3f}, p-value={pvalue:.2e}, graus de liberdade (dof) = {dof}"

In [ ]:
df

In [ ]:
expected.round(2)

In [ ]:
scales - expected

### Se a distribuição observada (df) similar à calculada ~ erro próximo de zero
  - Quanto menor o erro maior a área o rabo da distribuiçao estatística do chi2

### Valores esperados não se distanciam muito dos observados?

  - valores próximos: então as distribuições são similares:  aceita-se H0  --> p-value >= 0.05
  - valores distantes: então as distribuições NÃO são similares: rejeita-se Ha --> p-value < 0.05

### Importante o teste de chi2 depende dos graus de liberade

  - matriz n x m
  - dof = (n-1) * (m-1)

https://en.wikipedia.org/wiki/Chi-squared_test

In [ ]:
pvalue

In [ ]:
if pvalue >= 0.05:
    stri = "Aceita-se H0: distribuições estatisticamente similares"
else:
    stri = "Rejeita-se H0: distribuições estatisticamente diferentes"

stri += f", estatística = {chi2:.2f}, p-valor = {pvalue:.2e}"
stri

### A tabela de contingência é de 5 linhas x 2 colunas

In [ ]:
nrow, ncol = scales.shape
nrow, ncol

In [ ]:
dof = (nrow-1) * (ncol-1)
dof

### Dados mais diferentes: distribuições distintas

In [ ]:
# scales = observed data
scales = np.array([[16, 18, 16, 14, 12], [10, 24, 27, 35, 40]]).T
scales

In [ ]:
df = pd.DataFrame(scales)
df.columns = ['alt1200', 'alt200']
df.index = [32, 33, 34, 35, 36]
df

In [ ]:
stat = stats.chisquare(scales, axis=None)
stat

In [ ]:
def chi2_text(df, verbose=True):
    chi2, pvalue, dof, expected = stats.chi2_contingency(df)

    erro = np.sum((df.to_numpy() - expected)**2)

    if pvalue >= 0.05:
        stri = f"Aceita-se H0: distribuições estatisticamente similares"
    else:
        stri = f"Rejeita-se H0: distribuições estatisticamente diferentes"

    stri += f": estatística = {chi2:.2f}, p-valor = {pvalue:.2e}"
        
    if verbose:
        stri += "\nErro total = %d\nGraus de liberdade (dof) = %d"%(erro, dof)
        print(stri)
    
    return chi2, pvalue, dof, expected, erro, stri

In [ ]:
chi2, p, dof, expected, erro, stri = chi2_text(df, verbose=True)

### Simulando um afastamento ...

In [ ]:
# scales = observed data
scales = np.array([[16, 18, 16, 14, 12], [10, 24, 27, 35, 40]]).T
df = pd.DataFrame(scales)
df.columns = ['alt1200', 'alt200']
df.index = [32, 33, 34, 35, 36]
df

In [ ]:
df.alt200 = df.alt200 + 1
df

### De acordo que a distribuição esperada se afasta da observada o erro aumenta e o p-valor diminui

In [ ]:
# scales = observed data
scales = np.array([[16, 18, 16, 14, 12], [10, 24, 27, 35, 40]]).T
df = pd.DataFrame(scales)
df.columns = ['alt1200', 'alt200']
df.index = [32, 33, 34, 35, 36]

for i in range(5):
    print(f'{i+1})')
    chi2, p, dof, expected, erro, stri = chi2_text(df, verbose=True)
    df.alt200 = df.alt200 - 2
    print()
    

### Se eu tiver 3 altitude (0m, 200m, 1200m)
### Grau de liberdade = (n-1) * (m-1)
### Erro tende a aumentar mas o grau de liberdade também

In [ ]:
# scales = observed data
scales = np.array([[16, 18, 16, 14, 12], [10, 24, 27, 35, 40]]).T
print(scales)

df = pd.DataFrame(scales)
df.columns = ['alt1200', 'alt200']

chi2, p, dof, expected, erro, stri = chi2_text(df, verbose=True)

In [ ]:
# scales = observed data
scales = np.array([[16, 15, 14, 12, 15], [16, 18, 16, 14, 12], [10, 24, 27, 35, 40]]).T
print(scales)

df = pd.DataFrame(scales)
df.columns = ['alt0', 'alt1200', 'alt200']

chi2, p, dof, expected, erro, stri = chi2_text(df, verbose=True)

### Distribuição chi-quadrado

In [ ]:
# 2 altitudes
scales = np.array([[16, 18, 16, 14, 12], [10, 24, 27, 35, 40]]).T
df1 = pd.DataFrame(scales)
df1.columns = ['alt1200', 'alt200']

# 3 altitudes
scales = np.array([[16, 15, 14, 12, 15], [16, 18, 16, 14, 12], [10, 24, 27, 35, 40]]).T
df2 = pd.DataFrame(scales)
df2.columns = ['alt0', 'alt1200', 'alt200']

# 5 altitudes
scales = np.array([[16, 18, 16, 14, 10], 
                   [17, 19, 16, 13, 15],
                   [32, 24, 18, 11, 12],
                   [16, 18, 12, 16, 12],
                   [20, 18, 18, 16, 11]]).T
df3 = pd.DataFrame(scales)
df3.columns = ['alt0', 'alt1', 'alt2', 'alt1200', 'alt200']


In [ ]:
N = 300
colors = ['blue', 'green', 'red']

fig, ax = plt.subplots(figsize=(12,6))

for i in range(3):
    if   i == 0: df = df1
    elif i == 1: df = df2
    else:        df = df3
        
    color = colors[i]
    chi2, p, dof, expected, erro, stri = chi2_text(df, verbose=False)
    # print(i, dof, chi2, color)

    chi2_random_vals = np.random.chisquare(df=dof, size=N)
    label = f"dof={dof}"
    ax = sns.histplot(chi2_random_vals, stat='density', color=color, alpha=.2, label=label, ax=ax)

plt.grid()
plt.legend()
plt.title("Distribuição do Chi-quadrado")

In [ ]:
N = 300
colors = ['blue', 'green', 'red']

x_list = [2, 12, 25]

seqx = np.linspace(0, 40, 200)

fig, ax = plt.subplots(figsize=(12,6))

for i in range(3):
    if   i == 0: df = df1
    elif i == 1: df = df2
    else:        df = df3
        
    color = colors[i]
    chi2, p, dof, expected, erro, stri = chi2_text(df, verbose=False)
    # print(i, dof, chi2, color)

    chi2_random_vals = np.random.chisquare(df=dof, size=N)
    label = f"dof={dof}"
    chi2_pdf = stats.chi2.pdf(seqx, df=dof)
    sns.lineplot(x=seqx, y=chi2_pdf, color=color, label=label, ax=ax)

    x = x_list[i]
    y = stats.chi2.pdf(x, df=dof)
    pval = 1-stats.chi2.cdf(x, df=dof)

    ax.axvline(x=x, ymin=0, ymax=1, color=color)
    
    ax.annotate(f'chi2 stat {x} pdf={y:.1e} dof={dof}, pvalue={pval:.2f}', xy=(x,y), xytext=(x+0.3, y+0.01),
                color = color,
                arrowprops=dict(arrowstyle="->",
                                connectionstyle="angle3,angleA=0,angleB=-90", color=color))

plt.grid()
plt.legend()
plt.ylim(0, 0.22)
plt.title("Distribuição do Chi-quadrado");

### Adendo da distribuição de chi-quadrado

In [ ]:
mu = 0
sdv = 1
N = 1000

dist1 = np.random.normal(mu, sdv, N)**2
dist2 = np.random.normal(mu, sdv, N)**2

chi2_2 = dist1 + dist2

fig, ax = plt.subplots(figsize=(10,6))

color='blue'
ax = sns.histplot(dist1, stat='density', color=color, alpha=.2, label='normal**2', ax=ax)
sns.rugplot(dist1, color=color, alpha=0.4, ax=ax)

color='red'
ax = sns.histplot(chi2_2, stat='density', color=color, alpha=.2, label='chi2', ax=ax)
sns.rugplot(chi2_2, color=color, alpha=0.4, ax=ax)

# fit the chi2 distribution
from scipy import stats

df = 1
xseq = np.linspace(stats.chi2.ppf(0.01, df), stats.chi2.ppf(0.99, df), 100)
ax.plot(xseq, stats.chi2.pdf(xseq, df), color='navy')

df = 2
xseq = np.linspace(stats.chi2.ppf(0.01, df), stats.chi2.ppf(0.99, df), 100)
ax.plot(xseq, stats.chi2.pdf(xseq, df), color='darkred')

plt.ylim(0, 2)
plt.xlim(0,9)
plt.legend();

In [ ]:
dist3 = np.random.normal(mu, sdv, N)**2

chi2_3 = dist1 + dist2 + dist3

fig, ax = plt.subplots(figsize=(10,6))

color='blue'
ax = sns.histplot(dist1, stat='density', color=color, alpha=.2, label='normal**2', ax=ax)
sns.rugplot(dist1, color=color, alpha=0.4, ax=ax)

color='red'
ax = sns.histplot(chi2_2, stat='density', color=color, alpha=.2, label='chi2_2', ax=ax)
sns.rugplot(chi2_2, color=color, alpha=0.4, ax=ax)

color='green'
ax = sns.histplot(chi2_3, stat='density', color=color, alpha=.2, label='chi2_3', ax=ax)
sns.rugplot(chi2_3, color=color, alpha=0.4, ax=ax)

df = 1
xseq = np.linspace(stats.chi2.ppf(0.01, df), stats.chi2.ppf(0.99, df), 100)
ax.plot(xseq, stats.chi2.pdf(xseq, df), color='navy')

df = 2
xseq = np.linspace(stats.chi2.ppf(0.01, df), stats.chi2.ppf(0.99, df), 100)
ax.plot(xseq, stats.chi2.pdf(xseq, df), color='darkred')

df = 3
xseq = np.linspace(stats.chi2.ppf(0.01, df), stats.chi2.ppf(0.99, df), 100)
ax.plot(xseq, stats.chi2.pdf(xseq, df), color='darkgreen')

plt.ylim(0, 1)
plt.xlim(0,9)
plt.legend();